### Import nesssary model

In [1]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

### Load data

In [3]:
data = pd.read_csv("jester-data-1.csv", header=None)
#d = data.to_latex()
#text_file = open("Output_lab4_test.txt", "w")
#text_file.write(d)
#text_file.close()

In [4]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,...,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,...,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,...,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,...,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,...,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60


### Prepare the dataset

In [5]:
#user_ratings = data.values
user_ratings = data.values[0:100]

In [6]:
user_ratings_vali = np.zeros(user_ratings.shape) + 99.0
user_ratings_copy = user_ratings.copy() 
count = 0
for x in range(0,user_ratings.shape[0]):
    for y in range(0,user_ratings.shape[1]):
        if (user_ratings_copy[x,y]!=99.0):
            if (np.random.random() > 0.9):
                count += 1
                temp = user_ratings_copy[x,y]
                user_ratings_copy[x,y] = user_ratings_vali[x,y]
                user_ratings_vali[x,y] = temp
                #print(temp)
print(user_ratings_vali,'\n', count,'\n', count/(user_ratings.shape[0]*user_ratings.shape[1]))

[[99.   99.   99.   ... 99.   99.   99.  ]
 [99.   99.   99.   ... 99.   99.    1.07]
 [49.   99.   99.   ... 99.   99.   99.  ]
 ...
 [37.   99.   99.   ... 99.   99.   99.  ]
 [99.   99.   99.   ... 99.   99.   99.  ]
 [99.   99.   99.   ... 99.   99.   99.  ]] 
 716 
 0.07089108910891088


### Prepare the predicte matrix

In [7]:
n_features = 10
latent_user_preferences = np.random.random((user_ratings.shape[0], n_features))
latent_item_features = np.random.random((user_ratings.shape[1],n_features))

In [8]:
print('user_ratings.shape:           ',user_ratings.shape)
print('latent_user_preferences.shape:',latent_user_preferences.shape)
print('latent_item_features.shape:   ',latent_item_features.shape)

user_ratings.shape:            (100, 101)
latent_user_preferences.shape: (100, 10)
latent_item_features.shape:    (101, 10)


In [9]:
user_ratings, user_ratings.shape

(array([[ 74.  ,  -7.82,   8.79, ...,  99.  ,  99.  ,  99.  ],
        [100.  ,   4.08,  -0.29, ...,   0.34,  -4.32,   1.07],
        [ 49.  ,  99.  ,  99.  , ...,  99.  ,  99.  ,  99.  ],
        ...,
        [ 37.  ,  99.  ,  99.  , ...,  99.  ,  99.  ,  99.  ],
        [100.  ,   3.3 ,   1.31, ...,  -0.44,   1.17,   6.5 ],
        [ 37.  ,  99.  ,  99.  , ...,  99.  ,  99.  ,  99.  ]]), (100, 101))

In [10]:
np.random.random()

0.13755475885257595

### Training function

In [11]:
def predict_rating(user_id,item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)  #linear

def train(user_id, item_id, rating,alpha = 0.0001):
    
    #print (item_id)
    prediction_rating = predict_rating(user_id, item_id)
    err =  ( prediction_rating- rating );
    #print (err)
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    


def sgd(iterations = 100000):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    err_mse_log = 0
    for iteration in range(0,iterations):
        error = []
        for user_id in range(0,latent_user_preferences.shape[0]):
            for item_id in range(0,latent_item_features.shape[0]):
                rating = user_ratings_copy[user_id][item_id]
                if(rating != 99.0):
                    err = train(user_id,item_id,rating)
                    error.append(err)
                    #print(rating)
        mse = (np.array(error) ** 2).mean()
        #print(iteration)
        if(iteration%1000 == 0 ):
            err_mse = mse - err_mse_log
            err_mse_log = mse
            print (iteration, ':=', mse, err_mse)
                    
                    
    

In [12]:
sgd()

0 := 102.30180346764722 102.30180346764722
1000 := 9.595917991327335 -92.70588547631988
2000 := 9.134672057539275 -0.4612459337880601
3000 := 9.028114147209708 -0.10655791032956685
4000 := 8.973169409423905 -0.054944737785803
5000 := 8.933048600685513 -0.040120808738391744
6000 := 8.892632102963233 -0.04041649772227984
7000 := 8.847585531328326 -0.04504657163490755
8000 := 8.805828869983673 -0.04175666134465317
9000 := 8.774613700378291 -0.031215169605381732
10000 := 8.752412978163694 -0.022200722214597235
11000 := 8.735910025688087 -0.016502952475606847
12000 := 8.723168296670314 -0.012741729017772485
13000 := 8.713202285305572 -0.0099660113647424
14000 := 8.705386063315386 -0.007816221990186278
15000 := 8.699238968199403 -0.006147095115982992
16000 := 8.694372026576524 -0.00486694162287904
17000 := 8.690472286372662 -0.0038997402038614126
18000 := 8.687294667685572 -0.003177618687089989
19000 := 8.684653716352472 -0.002640951333100716
20000 := 8.682413479602733 -0.0022402367497385
21

KeyboardInterrupt: 

In [13]:
latent_user_preferences

array([[ 5.964e-01,  5.393e-01,  8.924e-03,  4.249e-01,  1.077e+00,
         2.537e-02,  2.737e-01,  1.572e+00, -1.927e+00,  3.888e+00],
       [ 4.596e-01,  1.592e-02, -3.088e-01,  1.288e+00,  9.453e-01,
         7.358e-01,  1.702e-01,  1.472e+00,  2.284e+00,  2.977e-01],
       [ 7.029e-01,  9.870e-02,  2.682e+00,  1.152e+00,  1.421e+00,
         4.819e-01,  1.168e+00,  1.818e+00,  1.086e+00, -1.066e+00],
       [ 1.772e+00, -6.514e-01,  1.158e+00,  1.953e-01,  1.012e+00,
         3.269e-01, -9.631e-01, -1.347e-01,  7.868e-01,  1.564e+00],
       [ 1.983e+00,  6.249e-01,  1.444e+00,  4.757e-02,  1.284e+00,
         5.765e-01,  3.602e-01,  1.131e+00,  3.257e-01,  3.799e-01],
       [ 9.870e-01,  2.201e+00, -1.054e+00,  9.787e-01, -2.617e-01,
         8.402e-01,  2.110e-02, -2.112e-01,  5.256e-01,  2.476e+00],
       [ 1.365e+00, -9.676e-02,  1.344e+00,  6.847e-01,  6.052e-01,
         4.828e-02,  9.284e-01,  7.552e-01, -2.831e-01, -9.613e-01],
       [ 1.665e+00,  1.927e+00,  2.804e-0

In [14]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[ 7.406e+01, -7.200e+00,  6.095e+00, ...,  1.467e+00,  1.279e+00,
        -4.056e+00],
       [ 1.004e+02,  3.118e+00,  1.858e-01, ...,  1.797e+00, -5.674e+00,
         2.551e+00],
       [ 1.042e+02,  8.710e+00,  8.677e+00, ...,  8.534e+00,  9.591e+00,
         6.511e+00],
       ...,
       [ 1.203e+03,  5.639e+01, -7.627e+00, ..., -2.196e+01, -1.430e+01,
        -1.298e+01],
       [ 9.925e+01,  3.598e+00,  7.352e-03, ..., -1.928e+00,  1.851e+00,
        -9.157e-02],
       [ 3.706e+01, -2.005e+00, -4.604e+00, ..., -6.252e-01, -6.718e+00,
        -3.503e+00]])

In [15]:
values = [zip(user_ratings_copy[i], predictions[i]) for i in range(0,predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = data.columns
comparison_data.applymap(lambda x: "(%2.3f|%2.3f)"%(x[0],x[1]))
#comparison_data.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,(74.000|74.059),(-7.820|-7.200),(8.790|6.095),(-9.660|-10.718),(99.000|-16.803),(-7.520|-6.539),(-8.500|-6.164),(-9.850|-8.793),(4.170|-1.065),(-8.980|-8.718),...,(99.000|-1.960),(99.000|-5.689),(99.000|3.402),(99.000|-4.121),(99.000|0.997),(99.000|-5.807),(-5.630|-3.178),(99.000|1.467),(99.000|1.279),(99.000|-4.056)
1,(100.000|100.354),(4.080|3.118),(-0.290|0.186),(6.360|4.832),(4.370|5.387),(-2.380|4.438),(-9.660|-0.644),(-0.730|-3.009),(-5.340|3.927),(8.880|5.380),...,(2.820|3.328),(-4.950|-2.902),(-0.290|1.608),(7.860|7.158),(-0.190|0.353),(-2.140|-2.288),(3.060|-0.414),(0.340|1.797),(-4.320|-5.674),(99.000|2.551)
2,(99.000|104.159),(99.000|8.710),(99.000|8.677),(99.000|5.704),(99.000|7.220),(9.030|7.154),(9.270|6.907),(9.030|8.903),(9.270|4.508),(99.000|7.065),...,(99.000|9.064),(99.000|6.485),(99.000|10.711),(9.080|8.639),(99.000|11.109),(99.000|6.062),(99.000|10.101),(99.000|8.534),(99.000|9.591),(99.000|6.511)
3,(48.000|48.129),(99.000|0.327),(8.350|5.235),(99.000|0.692),(99.000|-2.852),(1.800|6.787),(8.160|3.849),(-2.820|-3.769),(6.210|2.084),(99.000|-8.608),...,(99.000|0.900),(99.000|5.707),(99.000|-0.707),(0.530|-1.979),(99.000|9.071),(99.000|-1.531),(99.000|2.134),(99.000|7.957),(99.000|3.057),(99.000|5.082)
4,(91.000|91.029),(99.000|6.085),(4.610|4.548),(-4.170|4.051),(-5.390|-0.025),(1.360|3.563),(1.600|3.140),(7.040|0.164),(4.610|3.174),(-0.440|-0.920),...,(5.190|5.030),(99.000|5.192),(4.270|5.913),(5.190|2.226),(5.730|4.666),(99.000|4.900),(3.110|2.668),(99.000|3.649),(1.800|4.153),(1.600|4.384)
5,(100.000|99.554),(-6.170|-2.763),(-3.540|-6.454),(0.440|-3.316),(99.000|-7.611),(-7.090|-6.433),(-4.320|-6.814),(-8.690|-7.771),(-0.870|-1.013),(-6.650|-5.255),...,(-3.540|0.251),(-6.890|-5.761),(-0.680|-1.914),(-2.960|-1.791),(-2.180|-5.048),(-3.350|-4.605),(0.050|0.319),(-9.080|-4.610),(-5.050|-6.204),(-3.450|-5.816)
6,(47.000|47.464),(99.000|9.097),(99.000|4.301),(99.000|5.075),(99.000|4.773),(8.590|7.135),(-9.850|2.103),(7.720|9.128),(8.790|5.006),(99.000|3.129),...,(99.000|2.989),(99.000|3.256),(99.000|4.057),(99.000|5.529),(99.000|2.997),(2.330|4.208),(99.000|5.120),(99.000|0.118),(99.000|6.281),(99.000|1.821)
7,(100.000|100.031),(6.840|4.190),(3.160|1.231),(9.170|7.270),(99.000|-4.144),(-8.160|-6.735),(-1.700|-5.635),(99.000|8.503),(1.410|0.104),(99.000|-3.032),...,(7.230|3.955),(-1.120|-5.155),(-0.100|-0.012),(99.000|-11.124),(-3.160|-5.925),(-3.350|-3.251),(2.140|0.929),(-0.050|1.428),(1.310|-0.878),(0.000|1.546)
8,(99.000|89.699),(-3.790|-3.203),(-3.540|-3.135),(-9.420|-5.370),(-6.890|-5.232),(-8.740|-5.832),(-0.290|2.520),(-5.290|-5.516),(-8.930|-4.273),(-7.860|-3.874),...,(99.000|-1.103),(-0.290|0.485),(4.170|4.495),(-0.290|1.145),(-0.290|-2.633),(-0.290|2.089),(-0.290|-0.205),(-0.290|-0.069),(-3.400|-1.526),(-4.950|-0.913)
9,(72.000|71.762),(3.010|5.359),(5.150|5.918),(99.000|3.555),(3.010|4.095),(6.410|4.702),(5.150|4.612),(8.930|5.873),(2.520|2.794),(3.010|3.901),...,(99.000|4.175),(4.470|3.816),(99.000|6.565),(99.000|4.771),(99.000|6.032),(99.000|3.263),(99.000|5.142),(99.000|5.159),(99.000|5.596),(99.000|4.694)


In [16]:
values = [zip(user_ratings_vali[i], predictions[i]) for i in range(0,predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data_err = []
for x in range(0, comparison_data.shape[1]):
    for y in range(0, comparison_data.shape[0]):
        if (comparison_data[x][y][0] == 99.0):
            comparison_data[x][y] = (0,0)
        else:
            comparison_data_err.append(comparison_data[x][y][0] - comparison_data[x][y][1])
mse = (np.array(comparison_data_err) ** 2).mean()
comparison_data.columns = data.columns
comparison_data.applymap(lambda x: ["(%2.3f|%2.3f)"%(x[0],x[1]) ])

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(-8.160|-16.803)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],...,[(2.820|-1.960)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)]
1,[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],...,[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(1.070|2.551)]
2,[(49.000|104.159)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],...,[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)]
3,[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],...,[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)]
4,[(0.000|0.000)],[(8.500|6.085)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],...,[(0.000|0.000)],[(5.580|5.192)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(1.550|4.900)],[(0.000|0.000)],[(6.550|3.649)],[(0.000|0.000)],[(0.000|0.000)]
5,[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(-8.500|-7.611)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],...,[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)]
6,[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],...,[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)]
7,[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(-6.210|-4.144)],[(0.000|0.000)],[(0.000|0.000)],[(9.270|8.503)],[(0.000|0.000)],[(-5.190|-3.032)],...,[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(-5.680|-11.124)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)]
8,[(100.000|89.699)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],...,[(4.370|-1.103)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)]
9,[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(5.150|3.555)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],...,[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)],[(0.000|0.000)]


In [17]:
mse


2077.9401353703

In [ ]:
predictions

In [ ]:
latent_item_features

In [18]:
np.savetxt("predictions_10f_10.csv", predictions, delimiter=",")
np.savetxt("latent_user_preferences_10f_10.csv", latent_user_preferences, delimiter=",")
np.savetxt("latent_item_features_10f_10.csv", latent_item_features, delimiter=",")